In [8]:
import csv
import random
import datetime
allData = []
trainingData = []
testingData = []
columnName = []
SPLIT = 100 # means: out of 100, 70 will go to training and 30 will go to testing
random.seed(datetime.datetime.now())  # to get random values for every single call


In [9]:
def chance(percentage): #will generate 9 trues if percentage of success is given 90%
    if(random.randint(0,100) < percentage):
        return True
    else:
        return False

In [10]:
def readData(storage, columnName):
    columnName.clear()
    storage.clear()  #clearing the storage for better reruns
    fileName = "playgolf.csv"
    indexToIgnore = [0]  # ignoring the useless columns
    with open(fileName,encoding="utf8") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for i, rows in enumerate(csv_reader):
#             if(i>5000):
#                 break #breaking early just for quick runs
            tempData = []
            for j,data in enumerate(rows):
                if j not in indexToIgnore:
                    tempData.append(data)
            if i>0:
                storage.append(tempData)
            else:
                columnName.extend(tempData)

In [11]:
#reading all the data from the file and splitting them based on SPLIT chance
readData(allData, columnName)
trainingData.clear()
testingData.clear()
for data in allData:
    if(chance(SPLIT)):
        trainingData.append(data)
    else:
        testingData.append(data)

print("Data Description: \nAll: {}\nTraining: {}\nTesting: {}".format(len(allData), len(trainingData), len(testingData)))
print("Column Names: ", columnName)

Data Description: 
All: 14
Training: 14
Testing: 0
Column Names:  ['Outlook', 'Temperature', 'Humidity', 'Windy', 'Play Golf']


In [12]:
# training the data and storing the values in NB for testing later
nb = dict()  #our main data storage
allClassLabel = dict() #to store distinct set of class labels
labelIndex = len(trainingData[0])-1 #selecting the last row as label index, the index to be ignored

for rows in trainingData:
    for i, value in enumerate(rows):
        temp = (columnName[i], value, rows[labelIndex])  # constructing the tuple
        if i != labelIndex:
            if( nb.get( temp )):
                nb[ temp ] += 1
            else:
                nb[ temp ] = 1
        else:
            if(allClassLabel.get(value)):
                allClassLabel[value] += 1
            else:
                allClassLabel[value] = 1

# #DEBUG                
# for i in nb:
#     print(i, nb[i])
    
    
    
# nb has each row as ('home_country', 'Luxembourg', 'Loss')
'''
we have count of (values, output)
now we'll do count(value, output)/ count(output)
P( x | output ) = count(x^output) / count(output)
'''
for row in nb:
    # row = ('home_country', 'Scotland', 'Draw')
#     print(nb[row], allClassLabel[row[2]] )
    print(row, row[2], nb[row] , allClassLabel[row[2]])
    nb[row] = nb[row]/ allClassLabel[row[2]]
#     for j in i:
#         print(j)

# #DEBUG     
# for j in allClassLabel:
#     print(j, allClassLabel[j])

for output in allClassLabel:
    allClassLabel[output] = allClassLabel[output]/ len(trainingData)
    
    
#print("Classes: ", allClassLabel)
#print(nb)

# #DEBUG     
# for i in nb:
#     print(i, nb[i])

('Outlook', 'Rainy', 'No') No 3 5
('Temperature', 'Hot', 'No') No 2 5
('Humidity', 'High', 'No') No 4 5
('Windy', 'FALSE', 'No') No 2 5
('Windy', 'TRUE', 'No') No 3 5
('Outlook', 'Overcast', 'Yes') Yes 4 9
('Temperature', 'Hot', 'Yes') Yes 2 9
('Humidity', 'High', 'Yes') Yes 3 9
('Windy', 'FALSE', 'Yes') Yes 6 9
('Outlook', 'Sunny', 'Yes') Yes 3 9
('Temperature', 'Mild', 'Yes') Yes 4 9
('Temperature', 'Cool', 'Yes') Yes 3 9
('Humidity', 'Normal', 'Yes') Yes 6 9
('Outlook', 'Sunny', 'No') No 2 5
('Temperature', 'Cool', 'No') No 1 5
('Humidity', 'Normal', 'No') No 1 5
('Windy', 'TRUE', 'Yes') Yes 3 9
('Temperature', 'Mild', 'No') No 2 5
('Outlook', 'Rainy', 'Yes') Yes 2 9


In [13]:
def predict(datas, columnNames, rowToCheck, labels): # we are sending row without the label, and seeing what it predicts
    #print(rowToCheck)
#     print(columnNames)
#     print(labels)
    result = dict()
    for i in labels:
        result[i] = 1
    #print(result)
    
    for i, val in enumerate(rowToCheck):
        for j in labels:
            #print((columnNames[i],val, j))
            #('home_country', 'England', 'Draw') ('home_country', 'England', 'Win')
            if datas.get((columnNames[i],val, j)):
                result[j] *= datas[ (columnNames[i],val, j) ]
            else:
                result[j] = result[j] * 1.0 / allClassLabel[j] #laplace correction
    # now result has P(x|C)
    currMax = -10
    res = ""
    for i in result:
        result[i] *= allClassLabel[i]
        if(result[i] > currMax):
            currMax = result[i]
            res = i
    return res

correct = 0
for row in testingData:
    res = predict(nb, columnName, row[:-1], allClassLabel)
    if(res == row[-1]):
        correct += 1

incorrect = len(testingData) - correct
print("Correct Predictions: {}, Incorrect Predictions: {}".format(correct, incorrect))
# print( "Percentage of error: {}% ".format(int(incorrect/(len(testingData))*100 )))

Correct Predictions: 0, Incorrect Predictions: 0


In [14]:
print(trainingData[0])
customInput = ['Sunny', 'Hot', 'Normal', 'FALSE'] # notice that the customInput has no class label
output = predict(nb, columnName, customInput, allClassLabel)
print(output)

['Rainy', 'Hot', 'High', 'FALSE', 'No']
Yes
